In [192]:
from cluster import cluster as cl
rc = cl.start_cluster('kingcrab', 24)

In [193]:
rc.ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [194]:
dv = rc[:]

In [195]:
%%px --local

maxorbs = 1.e6

import numpy as np
import rebound
import random
import os
import time

mp = 5*3.e-6 # 5 earth masses
gamma = (2./3.*mp)**(1./3.) # Hill radius with factor of a taken out
a1 = 0.05 # semimajor axis of innermost planet in AU
imax = 1.*np.pi/180. # maximum inclination (uniform distribution)
emax = 2.e-2 # maximum eccentricity (uniform distribution)
betamin = 5.
betamax = 9.

In [196]:
def system(params):
    sim_id, dtfac = params
    random.seed(sim_id)
    
    beta = random.uniform(betamin, betamax)
    a2 = a1 + beta*gamma*a1
    a3 = a2 + beta*gamma*a2
    
    sim = rebound.Simulation()
    sim.integrator="whfast"
    sim.G = 4*np.pi**2
    sim.exit_min_distance = a1*gamma # use smaller hill radius as exit condition
    
    
    sim.add(m=1.)
    sim.add(m=mp, a=a1, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a2, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a3, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    
    P0 = ps[1].P
    tmax = maxorbs * P0 # number of inner planet orbital periods to integrate
    sim.dt = P0*dtfac
    
    features = [maxorbs, mp, gamma]
    for p in ps[1:]:
        features = features + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f, p.x, p.y, p.z, p.vx, p.vy, p.vz]
        
    stable = [True] # assume it's stable and update if not
    
    E0 = sim.calculate_energy()
    t0 = time.time()
    
    try:
        sim.integrate(tmax)
    except rebound.Encounter as error:
        stable = [False]
    
    features = stable+[sim.t/P0]+features+[np.abs((sim.calculate_energy()-E0)/E0), time.time()-t0]
    fname = '/mnt/raid-cita/dtamayo/Stability/data/whfastdt'+"{:.0e}".format(dtfac)+str(os.getpid())+'.csv'
    
    with open(fname, 'a') as f:
        f.write(str(sim_id))
        for feature in features:
            f.write(',{0}'.format(feature))
        f.write('\n')
        
    return features

In [197]:
Nsystems=2000
dtfacs = [0.08, 0.09, 0.2, 0.3]
res = []
for dtfac in dtfacs:
    params = list(zip(list(range(Nsystems)), [dtfac]*Nsystems)) 
    res.append(dv.map(system, params))

In [184]:
import pandas as pd
import numpy as np

columns = ['Sim_ID', 'Stable', 'Norbits_instability', 'Norbits', 'Mplanet', 'RH/a']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i, 'x'+i, 'y'+i, 'z'+i, 'vx'+i, 'vy'+i, 'vz'+i]
columns += ['Rel_Eerr', 'integ_time']

for dtfac in dtfacs:
    frames = []
    path = '/mnt/raid-cita/dtamayo/Stability/data/'
    for f in os.listdir(path):
            if f.startswith('whfastdt' + "{:.0e}".format(dtfac)):
                frames.append(pd.read_csv(path+f, names=columns, index_col=0))

    df = pd.concat(frames)
    df = df.sort_index()
    print("dtfac={0}, length={1}".format(dtfac, df.shape[0]))
    if df.shape[0] == Nsystems:
        df.to_csv(path+'1e6datasetwhfastdt'+"{:.0e}".format(dtfac)+'.csv', encoding='ascii')

dtfac=0.06, length=2000
dtfac=0.07, length=2000


In [93]:
cl.shutdown_cluster('homard', rc)